In [2]:
#21BAI1037 Fine tune the Dense neural network with various hyperparameters using grid search (MNIST dataset); Write inference about the results.

In [3]:
pip install -U keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.6 MB/s eta 0:00:00


In [4]:
import tensorflow as tf

In [5]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test,y_test) = mnist.load_data()
x_train, x_test = x_train/ 255.0, x_test / 255.0
print("x_train.shape:", x_train.shape)


11490434/11490434 [==============================] - 0s 0us/step
x_train.shape: (60000, 28, 28)


In [6]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

def build_model(hp):
  model = keras.Sequential()
  model.add(layers.Flatten(input_shape=(28,28)))

  model.add(layers.Dense(units=hp.Int('units',
                                      min_value = 32,
                                      max_value = 512,
                                      step = 128),
                         activation = 'relu'))
  model.add(layers.Dense(10,activation='softmax'))
  model.compile(
      optimizer=keras.optimizers.Adam(
          hp.Choice('learning_rate',values=[1e-2,1e-4])),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])
  return model



<ipython-input-6-a8e308a4ecc1>:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [7]:
# creating an instance for the random search class
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='lab3')



In [8]:
tuner.search_space_summary() # comparing 2 hyperparamters : no of units is a layer and learning rate


Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 128, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.0001], 'ordered': True}


In [9]:
tuner.search(x_train, y_train,      #works like model.fit
             epochs=5,
             validation_data=(x_test,y_test))

Trial 5 Complete [00h 03m 10s]
val_accuracy: 0.9619666735331217

Best val_accuracy So Far: 0.9660333395004272
Total elapsed time: 00h 14m 59s


In [10]:
tuner.results_summary()

Results summary
Results in my_dir/lab3
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
units: 416
learning_rate: 0.0001
Score: 0.9660333395004272

Trial 0 summary
Hyperparameters:
units: 160
learning_rate: 0.01
Score: 0.9621000091234843

Trial 4 summary
Hyperparameters:
units: 288
learning_rate: 0.0001
Score: 0.9619666735331217

Trial 1 summary
Hyperparameters:
units: 416
learning_rate: 0.01
Score: 0.9602333307266235

Trial 2 summary
Hyperparameters:
units: 160
learning_rate: 0.0001
Score: 0.9535666704177856


In [11]:
def build_model2(hp):
  model = tf.keras.Sequential()
  model.add(layers.Flatten(input_shape=(28,28)))
  for i in range (hp.Int('layers',2,6)):
    model.add(tf.keras.layers.Dense(
        units=hp.Int('units_'+str(i),50,100,step=10),
        activation=hp.Choice('act_'+str(i),['relu','sigmoid'])))
    model.add(tf.keras.layers.Dense(1, activation ='softmax'))
    model.compile('adam','binary_crossentropy',metrics=['accuracy'])
    return model

In [16]:
tuner2 = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir2',
    project_name='lab3')


In [17]:
tuner2.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 128, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.0001], 'ordered': True}


In [18]:
tuner2.search(x_train, y_train,      #works like model.fit
             epochs=5,
             validation_data=(x_test,y_test))

Trial 5 Complete [00h 03m 07s]
val_accuracy: 0.9630333185195923

Best val_accuracy So Far: 0.9630333185195923
Total elapsed time: 00h 15m 56s


In [19]:
tuner2.results_summary()

Results summary
Results in my_dir2/lab3
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 4 summary
Hyperparameters:
units: 288
learning_rate: 0.01
Score: 0.9630333185195923

Trial 2 summary
Hyperparameters:
units: 288
learning_rate: 0.0001
Score: 0.9622333447138468

Trial 3 summary
Hyperparameters:
units: 416
learning_rate: 0.01
Score: 0.9618666569391886

Trial 1 summary
Hyperparameters:
units: 160
learning_rate: 0.01
Score: 0.9608333309491476

Trial 0 summary
Hyperparameters:
units: 160
learning_rate: 0.0001
Score: 0.9553666710853577
